<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [ ]:
#检测使用的显卡

#注册https://wandb.ai/authorize 填api 获得实时监控网址
! /opt/bin/nvidia-smi
!pip install wandb
import wandb
wandb.init()

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

In [3]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



Saving 序列 02_1.mp4 to 序列 02_1.mp4
move 序列 02_1.mp4 to upload/序列 02_1.mp4


In [4]:
#Drive上传方式 需要授权
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

filename='[SHANA]videoplayback.mkv' #填文件名
dst_path1 = os.path.join("/content/drive/MyDrive/", filename)

shutil.copy(dst_path1,"/content/Real-ESRGAN/upload/")




'/content/Real-ESRGAN/upload/[SHANA]videoplayback.mkv'

In [38]:
# 检测输入文件FPS。
tmp_path = os.path.join("/content/Real-ESRGAN/upload/", filename)
import cv2

video = cv2.VideoCapture(tmp_path)

# fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
#        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))

fps = video.get(cv2.CAP_PROP_FPS)
print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

##capture = cv2.VideoCapture.get(tmp_path)
#cap = cv2.VideoCapture(f'/content/upload/{filename}')

#frame_count = capture.get(5)
#fps = int(round(cap.get(cv2.CAP_PROP_FPS)))
#fps = cap.get(cv2.CAP_PROP_FPS)
print(f'fps= {fps}')


Frames per second using video.get(cv2.CAP_PROP_FPS) : 0.0
fps= 0.0


In [ ]:
#更新ffmpeg 
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!apt-get update
!apt-get install ffmpeg

In [ ]:
shell ffmpeg -version

In [43]:
%shell ffmpeg -i '/content/Real-ESRGAN/upload/{filename}' -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 1000,2000,3000,4000,5000 out%03d.mkv

SyntaxError: ignored

In [ ]:
!apt-get install nodejs-dev node-gyp libssl1.0-dev
!apt-get install npm
!$ npm install --global ffmpeg-progressbar-cli


In [ ]:
! npm install --global ffmpeg-progressbar-cli

In [ ]:
#拆帧
filename='[SHANA]videoplayback.mkv'
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
%shell ffmpeg -hwaccel_device 0 -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


# 3. Inference


In [ ]:
#超分
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

Testing 0 00000001
	Tile 1/390
	Tile 2/390
	Tile 3/390
	Tile 4/390
	Tile 5/390
	Tile 6/390
	Tile 7/390
	Tile 8/390
	Tile 9/390
	Tile 10/390
	Tile 11/390
	Tile 12/390
	Tile 13/390
	Tile 14/390
	Tile 15/390
	Tile 16/390
	Tile 17/390
	Tile 18/390
	Tile 19/390
	Tile 20/390
	Tile 21/390
	Tile 22/390
	Tile 23/390
	Tile 24/390
	Tile 25/390
	Tile 26/390
	Tile 27/390
	Tile 28/390
	Tile 29/390
	Tile 30/390
	Tile 31/390
	Tile 32/390
	Tile 33/390
	Tile 34/390
	Tile 35/390
	Tile 36/390
	Tile 37/390
	Tile 38/390
	Tile 39/390
	Tile 40/390
	Tile 41/390
	Tile 42/390
	Tile 43/390
	Tile 44/390
	Tile 45/390
	Tile 46/390
	Tile 47/390
	Tile 48/390
	Tile 49/390
	Tile 50/390
	Tile 51/390
	Tile 52/390
	Tile 53/390
	Tile 54/390
	Tile 55/390
	Tile 56/390
	Tile 57/390
	Tile 58/390
	Tile 59/390
	Tile 60/390
	Tile 61/390
	Tile 62/390
	Tile 63/390
	Tile 64/390
	Tile 65/390
	Tile 66/390
	Tile 67/390
	Tile 68/390
	Tile 69/390
	Tile 70/390
	Tile 71/390
	Tile 72/390
	Tile 73/390
	Tile 74/390
	Tile 75/390
	Tile 76/390
	T

# 5. Download Results


In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>